In [9]:
from pandas import read_pickle
from tqdm import tqdm
from thefuzz import process, fuzz

from dotenv import load_dotenv
from os import getenv
load_dotenv()


API_KEY = getenv('API_KEY')

In [26]:
df = read_pickle('./data/api_request_results/retrieved_data.zip').reset_index(drop=True)


In [27]:
df['simi_score'] = [fuzz.partial_ratio(row.title, row.db_title) for row in df.itertuples()]
df.sort_values(['db_id','simi_score'], inplace=True)
df.drop_duplicates(['db_id'], keep='last', inplace=True)
df.reset_index(drop=True, inplace=True)

In [24]:
# Realmente queremos quedarnos unicamente con los papers del dataset?
df_filter = df.query('simi_score>80') # hay falsos positivos, pero no muchos

# O unicamente con los papers que son de acceso abierto (pdf descargable)
df = df[df.isOpenAccess].reset_index(drop=True)

# usar todo => más datos!


In [ ]:
## Recuperar los datos para referencias y citas